In [1]:
using Pkg; Pkg.activate("C:\\Users\\KarlH\\Desktop\\ComputationalPhysicsMaster\\Code\\OrbitalNeuralMethods")
using Revise
using OrbitalNeuralMethods

  Activating project at `C:\Users\KarlH\Desktop\ComputationalPhysicsMaster\Code\OrbitalNeuralMethods`
┌ Info: Precompiling OrbitalNeuralMethods [6de8678a-dd1c-461e-a987-24da1d9562b4]
└ @ Base loading.jl:1662


In [2]:
import Random
rng = Random.MersenneTwister();

In [13]:
n = 4
ω = 0.25
ham = HOCoulomb(ω, shielding = 0.25);

l = 20
basis = SpinBasis(HOBasis(l, ω))
grid = [x for x in range(-10, stop = 10, length = 2001)]
system = System(n, basis, grid, ham);

mixer = Alpha(0.5)
rhf = RHF(system, mixer);
rhf = compute_ground_state!(rhf, max_iter = 1000, tol = 1e-10, verbose=0);
energy(rhf)

4.466702608698696

In [21]:
metro = Importance(equils = 1e4, samples = 1e6, step = 0.01);
#metro = Metropolis(equils = 1e4, samples = samples, step = 0.1);

In [15]:
nn = NeuralNetwork([Dense(48), Tanh, Dense(8), Tanh, Dense(1), Exp], n=n, rng = rng);
slater = Slater(rhf);
wf = SlaterNN(slater, nn);
optimizer = ADAM(wf, lr = 0.01, max_iter = 10, tol = 1e-6);

In [25]:
@time wf, grad_results = optimize(wf, ham, metro, optimizer, nthreads = 12, verbose = true);

E = 3.959 iter = 10/10                                      
No convergence reached, final norm of gradient was 0.03886404413895812
 15.586961 seconds (35.43 k allocations: 16.735 MiB)


In [26]:
samples_b = 2^24
metro_b = Importance(equils = 1e5, samples = samples_b, step = 0.1);
@time result = blocking(wf, ham, metro_b, nthreads = 12);
result.E, samples_b

In [ ]:
#nn = NeuralNetwork([Dense(24), Tanh, Dense(8), Tanh, Dense(1), Exp], n=n, rng = rng);
#3.8633600652775524


#3.8657079926602127